# Gas Main Investment Cost and Risk modelling
Results of the machine learning models that predict cost and associated risk with the investment in gas mains in Poland. Models output the expected risk and cost of the specified main relatively to the price of one kilometer of the most typical main (standard model, 700mm diameter, 4.4MPa pressure).

### Preparing models
Models that will be used are specified `gmie/config.py` file. All new models first need to be trained as specifed in the `README.md`

In [1]:
import logging
import numpy as np
import tensorflow as tf

from gmie.config import *
from gmie.preprocessing import *


logging.getLogger('tensorflow').setLevel(logging.WARNING)
all_models = TRAINING_MODELS + SOLVABLE_MODELS
collections = [(model.name,
                tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=model.name))
               for model in all_models]
savers = [(name, tf.train.Saver(var_list=coll)) for name, coll in collections]

sess = tf.Session()
sess.run(tf.global_variables_initializer())
[s.restore(sess, os.path.join(SAVE_DIR, name)) for name, s in savers]

[None, None]

In [2]:
from ipywidgets import widgets as iws
from ipywidgets.widgets import interact
from matplotlib import pyplot as plt
from scipy import stats

from gmie.mains import MainModel


def plot_gaussian(mean, variance):
    sigma = math.sqrt(variance)
    x = np.linspace(mean - 3*sigma, mean + 3*sigma, 100)
    plt.plot(x, stats.norm.pdf(x, mean, sigma))
    plt.show()

### Run models and visualise outputs

In [5]:
@interact(
    diameter=iws.IntSlider(value=750, min=100, max=1400, 
                           step=5, description='Diameter:'),
    model=iws.SelectionSlider(options=['reduced', 'standard', 'elevated'],
                              value='standard', description='Model:'),
    pressure=iws.FloatSlider(value=4.4, min=2.0, max=7.0, 
                             step=0.01, description='Pressure')
)
def run_model(diameter, model, pressure):
    model = MainModel[model.upper()]
    inputs = np.array([[diameter, model.value, pressure]])
    for model in all_models:
        norm_inputs = model.normalize_inputs(inputs, sess)
        output = model.predict(norm_inputs, sess)
        denorm_output = model.denormalize_outputs(output, sess)[0][0]
        variance = model.get_denorm_variance(sess)[0]
        print(f'{model.name}:')
        print('mean:', denorm_output)
        print('variance:', variance)
        plot_gaussian(denorm_output, variance)

interactive(children=(IntSlider(value=750, description='Diameter:', max=1400, min=100, step=5), SelectionSlide…